# Painter Classification Example Code #

Let's define all imports here

In [1]:
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

import utils
from generator import Img_DataGenerator
import model_functions_new

Using TensorFlow backend.


Let's define out model hyperparameters here

In [2]:
load_pretrained_model = False
train_top_only = True

train_top = True

model_path = ''

set_name_list = ['Train', 'Val', 'Test']
batch_size = 16
nr_labels = 3


# shuffle currently NEEDS to be False (otherwise prediction does'nt work)
# TODO: Sort out data generator
params = {'dim': (224,224),
          'batch_size': batch_size,
          'n_classes': nr_labels,
          'n_channels': 3,
          'shuffle': True}

# Hyperparameters

# Dropout rate
dor=0.3

# Number of layers to freeze - denotes the index of the first UNFROZEN layer
# NOTE: If only a top-model is trained, set train_top_model(above) to True
# and leave this variable at 0. Only used if SOME convolutional layers are
# frozen during the training process
num_to_freeze = 50

# PATHS TO THE SAVED MODELS
# Save last model to the following path
last_model_path = 'Models/last_model'

# Save best model during training to the following path
callback_path = "Models/res50_{epoch:.0f}_{val_acc:.4f}_{acc:.4f}"

# The list of sub directories for the training, validation and test images 
set_name_list = ['Train', 'Val', 'Test']

batch_size = 32

epochs = 100

learning_rate = 1e-5

# Define optimizer
optim = optimizers.Adam(lr=learning_rate)
monitor_metric = 'val_acc'

# Set up the exact call to the model you want as a string
model_string = 'model_functions_new.cust_res50_classifier(nr_labels, dor)'

Sett up the generators for training, validation and test data

In [3]:
# The directory where the images are stored
img_data_directory = ''

# The lists for paths to training, validation and test data 
train_data_list, val_data_list, test_data_list, labels_dict = \
utils.get_img_data(img_data_directory, set_name_list)

Set up the generators

In [4]:
def get_img_data_generator(id_list):
    return Img_DataGenerator(id_list, labels_dict, **params)


train_generator = get_img_data_generator(train_data_list)
val_generator = get_img_data_generator(val_data_list)
test_generator = get_img_data_generator(test_data_list)

Set up the model

In [5]:
if load_pretrained_model == False:
    model = eval(model_string)
else:
    model = load_model(model_path)

model.summary()

# Make sure all layers of res50 are trainable (even when pretrained model, i.e. top-model is loaded)
for count, layer in enumerate(model.layers[1].layers):
        layer.trainable = True

# Freeze layers if required
if num_to_freeze != 0:
    for count, layer in enumerate(model.layers[1].layers):
        if count < num_to_freeze:
            layer.trainable = False

# Freeze all convolutional layers if required (training top-model only)            
if train_top_only == True:
    # Freeze all convolutional layers
    for layer in model.layers[1].layers:
        layer.trainable = False
        
if train_top == False:
    for layer in model.layers[2].layers:
        layer.trainabel = False
        
# compile the model with specified optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
        
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 7, 2048)        0         
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 6147      
Total params: 4,202,499
Trainable params: 4,202,499
Non-trainable params: 0
_________________________________________________________________


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
model_1 (Model)              (None, 3)                 4202499   
Total params: 27,790,211
Trainable params: 27,737,091
Non-trainable params: 53,120
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
___________________________________________________________

/home/nbuser/library/model_functions_new.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`
  final_model = Model(inputs=inp, output = y)


In [6]:
# Set up empty lists for plotting
lossList = []
valLossList = []
accList = []
valAccList = []
lrList = []

# Save best model accordig to chosen metric
checkpoint = ModelCheckpoint(callback_path, monitor=monitor_metric,
                             save_best_only=True, save_weights_only=False, 
                             mode='auto', period=1, verbose=1)

Run model

In [7]:
history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=epochs,
                              callbacks=[checkpoint], 
                              verbose=1)

# Put plotting metrics in respective list
lossList.extend(history.history['loss'])
valLossList.extend(history.history['val_loss'])
accList.extend(history.history['acc'])
valAccList.extend(history.history['val_acc'])

loss, acc = model.evaluate_generator(test_generator, verbose=1)

print('Accuracy on test set: ' + str(acc))

Epoch 1/100
140/140 [==============================] - 856s 6s/step - loss: nan - acc: 0.8924 - val_loss: nan - val_acc: 0.9062

Epoch 00001: val_acc improved from -inf to 0.90625, saving model to Models/res50_1_0.9062_0.8924


OSError: Unable to create file (unable to open file: name = 'Models/res50_1_0.9062_0.8924', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)